## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-26-20-51-22 +0000,nypost,Striking nurses and NYC hospitals clear ‘major...,https://nypost.com/2026/01/26/us-news/striking...
1,2026-01-26-20-50-47 +0000,nyt,"As U.S. Warships Get Closer, Iran Ramps Up Thr...",https://www.nytimes.com/2026/01/26/world/middl...
2,2026-01-26-20-49-59 +0000,nypost,Bill Ackman gives $10K to family of Alex Prett...,https://nypost.com/2026/01/26/business/bill-ac...
3,2026-01-26-20-48-28 +0000,nypost,Holocaust Museum scolds Tim Walz over ‘deeply ...,https://nypost.com/2026/01/26/us-news/holocaus...
4,2026-01-26-20-48-00 +0000,wsj,Treasury Yields Fall Amid Political Tension,https://www.wsj.com/finance/jgbs-rise-as-yens-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2382/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
35,trump,44
25,minneapolis,25
26,shooting,24
230,minnesota,22
139,ice,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
259,2026-01-26-01-57-16 +0000,nypost,Trump ‘reviewing everything’ about fatal shoot...,https://nypost.com/2026/01/25/us-news/trump-re...,148
182,2026-01-26-12-22-24 +0000,nypost,Powerful House Republican suggests Trump consi...,https://nypost.com/2026/01/26/us-news/powerful...,148
185,2026-01-26-12-03-00 +0000,wsj,Republican unity over President Trump’s immigr...,https://www.wsj.com/politics/policy/miller-noe...,118
28,2026-01-26-20-03-14 +0000,nyt,Trump Pivots From Attacks on Walz Amid Growing...,https://www.nytimes.com/2026/01/26/us/politics...,112
143,2026-01-26-15-14-23 +0000,bbc,Trump says administration 'reviewing everythin...,https://www.bbc.com/news/articles/cr571qg4m61o...,112


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
259,148,2026-01-26-01-57-16 +0000,nypost,Trump ‘reviewing everything’ about fatal shoot...,https://nypost.com/2026/01/25/us-news/trump-re...
102,67,2026-01-26-17-16-14 +0000,startribune,Live: Trump speaks with Walz as judge holds he...,https://www.startribune.com/ice-raids-minnesot...
34,57,2026-01-26-19-56-06 +0000,wapo,Six presumed dead in private jet crash at Main...,https://www.washingtonpost.com/transportation/...
281,49,2026-01-25-22-48-00 +0000,wsj,Senate Democrats angered by the deadly shootin...,https://www.wsj.com/politics/policy/senate-dem...
73,43,2026-01-26-18-26-00 +0000,wsj,Israel Says It Will Reopen Gaza Border Crossin...,https://www.wsj.com/world/middle-east/israel-t...
229,39,2026-01-26-08-36-39 +0000,wsj,Stock Market Today: Dow Rises; Gold Price Tops...,https://www.wsj.com/livecoverage/stock-market-...
161,34,2026-01-26-14-24-51 +0000,nypost,Trump sending border czar Tom Homan to Minneso...,https://nypost.com/2026/01/26/us-news/trump-se...
14,29,2026-01-26-20-32-47 +0000,latimes,A gunman is spotted near a grade school. O.C. ...,https://www.latimes.com/california/story/2026-...
22,28,2026-01-26-20-18-35 +0000,nypost,Trump says he’s ‘amazed’ Gavin Newsom’s runni...,https://nypost.com/2026/01/26/us-news/trump-bl...
287,26,2026-01-25-22-14-46 +0000,nypost,Cash-strapped Washington Post could fold entir...,https://nypost.com/2026/01/25/media/washington...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
